# P3 - Movie Genre Classification

Elaborado por GRUPO 1:

- Juanita Piraban Barbosa - 201216313
- Lorena Morales Rodríguez - 202027957
- Alejandro Barinas Guio - 201628859
- Jaime Humberto Trujillo Perea - 201920366
- Alexander Zapata Galindo - 201425426

### Objective

Classify a movie genre based on its plot.
Source: https://www.kaggle.com/c/miia4201-202019-p3-moviegenreclassification/overview

- Input: movie plot
- Output: Probability of the movie belong to each genre

### Evaluation

- 50% Report with all the details of the solution, the analysis and the conclusions. The report cannot exceed 10 pages, must be send in PDF format and must be self-contained.
- 50% Performance in the Kaggle competition (The grade for each group will be proportional to the ranking it occupies in the competition. The group in the first place will obtain 5 points, for each position below, 0.25 points will be subtracted, that is: first place: 5 points, second: 4.75 points, third place: 4.50 points ... eleventh place: 2.50 points, twelfth place: 2.25 points).

### Deatline

- The project must be carried out in the groups assigned.
- Use clear and rigorous procedures.
- The delivery of the project is on August 1st, 2021, 11:59 pm, through Bloque Neón.
- No projects will be received after the delivery time or by any other means than the one established.

### Acknowledgements

We thank Professor Fabio Gonzalez, Ph.D. and his student John Arevalo for providing this dataset.
See https://arxiv.org/abs/1702.01992

## Sample Submission

In [1]:
# Librerías

import pandas as pd
from matplotlib import pyplot as plt
import os
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.metrics import r2_score, roc_auc_score
from sklearn.model_selection import train_test_split

from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import SVC
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score

from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split

from sklearn.metrics import accuracy_score
from sklearn.metrics import make_scorer

import nltk
from nltk.corpus import stopwords

In [2]:
df_train = pd.read_csv('DataSet/dataTraining.csv', encoding='UTF-8', index_col=0)
df_test = pd.read_csv('Dataset/dataTesting.csv', encoding='UTF-8', index_col=0)

In [3]:
df_train.head()

,year,title,plot,genres,rating
3107,2003,Most,most is the story of a single father who takes...,"['Short', 'Drama']",8.0
900,2008,How to Be a Serial Killer,a serial killer decides to teach the secrets o...,"['Comedy', 'Crime', 'Horror']",5.6
6724,1941,A Woman's Face,"in sweden , a female blackmailer with a disfi...","['Drama', 'Film-Noir', 'Thriller']",7.2
4704,1954,Executive Suite,"in a friday afternoon in new york , the presi...",['Drama'],7.4
2582,1990,Narrow Margin,"in los angeles , the editor of a publishing h...","['Action', 'Crime', 'Thriller']",6.6


In [4]:
df_train['rating'] = df_train['rating'].astype(int)
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7895 entries, 3107 to 215
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   year    7895 non-null   int64 
 1   title   7895 non-null   object
 2   plot    7895 non-null   object
 3   genres  7895 non-null   object
 4   rating  7895 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 370.1+ KB


In [5]:
df_test.head()

,year,title,plot
1,1999,Message in a Bottle,"who meets by fate , shall be sealed by fate ...."
4,1978,Midnight Express,"the true story of billy hayes , an american c..."
5,1996,Primal Fear,martin vail left the chicago da ' s office to ...
6,1950,Crisis,husband and wife americans dr . eugene and mr...
7,1959,The Tingler,the coroner and scientist dr . warren chapin ...


In [6]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3383 entries, 1 to 11275
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   year    3383 non-null   int64 
 1   title   3383 non-null   object
 2   plot    3383 non-null   object
dtypes: int64(1), object(2)
memory usage: 105.7+ KB


### Create y

In [7]:
df_train['genres'] = df_train['genres'].map(lambda x: eval(x))

le = MultiLabelBinarizer()
y_genres = le.fit_transform(df_train['genres'])

In [8]:
y_genres

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 1, 0, 0],
       ...,
       [0, 1, 0, ..., 0, 0, 0],
       [0, 1, 1, ..., 0, 0, 0],
       [0, 1, 1, ..., 0, 0, 0]])

### Create count vectorizer


In [9]:
english_stopwords = stopwords.words('english')

vect = CountVectorizer(max_features=1000 , stop_words=english_stopwords, ngram_range=(1, 1))
X_dtm = vect.fit_transform(df_train['plot'])

In [10]:
print(vect.get_feature_names())

['abandoned', 'able', 'accepts', 'accident', 'accidentally', 'across', 'act', 'action', 'actor', 'actress', 'actually', 'adam', 'adult', 'adventure', 'affair', 'age', 'aged', 'agent', 'agents', 'ago', 'agrees', 'air', 'alan', 'alex', 'alice', 'alien', 'alive', 'almost', 'alone', 'along', 'already', 'also', 'although', 'always', 'america', 'american', 'among', 'ancient', 'andy', 'angeles', 'ann', 'anna', 'annie', 'another', 'anyone', 'anything', 'apart', 'apartment', 'appears', 'area', 'army', 'around', 'arrested', 'arrival', 'arrive', 'arrives', 'art', 'arthur', 'artist', 'asks', 'assigned', 'assistant', 'attack', 'attacked', 'attempt', 'attempts', 'attention', 'attorney', 'attracted', 'aunt', 'away', 'baby', 'back', 'bad', 'band', 'bank', 'bar', 'based', 'battle', 'beautiful', 'become', 'becomes', 'becoming', 'befriends', 'begin', 'beginning', 'begins', 'behavior', 'behind', 'believe', 'believes', 'beloved', 'ben', 'best', 'better', 'beyond', 'big', 'bill', 'billy', 'birthday', 'black

### Train / Test DataSets

In [11]:
X_train, X_test, y_train_genres, y_test_genres = train_test_split(X_dtm, y_genres, test_size=0.33, random_state=1234)

In [12]:
X_train

<5289x1000 sparse matrix of type '<class 'numpy.int64'>'
	with 150580 stored elements in Compressed Sparse Row format>

In [13]:
X_test

<2606x1000 sparse matrix of type '<class 'numpy.int64'>'
	with 73787 stored elements in Compressed Sparse Row format>

In [14]:
y_train_genres

array([[0, 1, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 1, 0, 0],
       [0, 0, 0, ..., 1, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [15]:
y_test_genres

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 1, 0, 0],
       [1, 0, 0, ..., 1, 0, 0],
       [1, 0, 0, ..., 1, 0, 0]])

### Initial multi-class / multi-label model

In [16]:
clf = OneVsRestClassifier(RandomForestClassifier(n_jobs=-1, n_estimators=100, max_depth=None, random_state=1234))
clf.fit(X_train, y_train_genres)

OneVsRestClassifier(estimator=RandomForestClassifier(n_jobs=-1,
                                                     random_state=1234))

In [17]:
y_pred_genres = clf.predict_proba(X_test)
roc_auc_score(y_test_genres, y_pred_genres, average='macro')

0.7534141069668597

### Tuning model

In [34]:
# hiperparameters
n_estimators = [100]
random_state = [1234]
bootstrap = ['True']    
max_features = ['auto', 'sqrt'] # Number of features to consider at every split
max_depth = [10, 15, 20, 25] #  Maximum number of levels in tree[2, 5, 10, 15, 'None']
min_samples_split = [2, 5, 7, 10] # Minimum number of samples required to split a node[2, 3, 4, 5]
min_samples_leaf = [1, 5, 7, 10] # Minimum number of samples required at each leaf node[1, 2, 3, 4, 5]
parameters = {
               'estimator__n_estimators':n_estimators,
               'estimator__random_state':random_state,
               'estimator__bootstrap':bootstrap,
               'estimator__max_features': max_features,
               'estimator__max_depth': max_depth,
               'estimator__min_samples_split': min_samples_split,
               'estimator__min_samples_leaf': min_samples_leaf
            }

# model_to_set
clf = OneVsRestClassifier(RandomForestClassifier())

# model_to_tune
tuning = GridSearchCV(estimator=clf, param_grid=parameters, cv=10, n_jobs=-1, refit='roc_auc_score')
tuning.fit(X_train, y_train_genres)

GridSearchCV(cv=10,
             estimator=OneVsRestClassifier(estimator=RandomForestClassifier()),
             n_jobs=-1,
             param_grid={'estimator__bootstrap': ['True'],
                         'estimator__max_depth': [10, 15, 20, 25],
                         'estimator__max_features': ['auto', 'sqrt'],
                         'estimator__min_samples_leaf': [1, 5, 7, 10],
                         'estimator__min_samples_split': [2, 5, 7, 10],
                         'estimator__n_estimators': [100],
                         'estimator__random_state': [1234]},
             refit='roc_auc_score')

In [36]:
# best parameters
best_param = tuning.best_params_
best_param

{'estimator__bootstrap': 'True',
 'estimator__max_depth': 25,
 'estimator__max_features': 'auto',
 'estimator__min_samples_leaf': 1,
 'estimator__min_samples_split': 5,
 'estimator__n_estimators': 100,
 'estimator__random_state': 1234}

In [18]:
best_param_1={'estimator__bootstrap': 'True',
 'estimator__max_depth': 25,
 'estimator__max_features': 'auto',
 'estimator__min_samples_leaf': 1,
 'estimator__min_samples_split': 5,
 'estimator__n_estimators': 100,
 'estimator__random_state': 1234}

In [19]:
# best_model
best_model = OneVsRestClassifier(RandomForestClassifier(n_jobs=-1, n_estimators=100, bootstrap=True, random_state=1234,
                                                max_features=best_param_1['estimator__max_features'],
                                                max_depth=best_param_1['estimator__max_depth'],
                                                min_samples_split=best_param_1['estimator__min_samples_split'],
                                                min_samples_leaf=best_param_1['estimator__min_samples_leaf']))
best_model.fit(X_train, y_train_genres)

OneVsRestClassifier(estimator=RandomForestClassifier(max_depth=25,
                                                     min_samples_split=5,
                                                     n_jobs=-1,
                                                     random_state=1234))

In [20]:
# ROC_AUC best training model
y_pred_genres = best_model.predict_proba(X_test)
roc_auc_score(y_test_genres, y_pred_genres, average='macro',multi_class='ovr')

0.7645981877617919

### Fit over all train dataset

In [21]:
# final_model
final_model = OneVsRestClassifier(RandomForestClassifier(n_jobs=-1, n_estimators=100, bootstrap=True, random_state=1234,
                                                max_features=best_param_1['estimator__max_features'],
                                                max_depth=best_param_1['estimator__max_depth'],
                                                min_samples_split=best_param_1['estimator__min_samples_split'],
                                                min_samples_leaf=best_param_1['estimator__min_samples_leaf']))
final_model.fit(X_dtm, y_genres)

OneVsRestClassifier(estimator=RandomForestClassifier(max_depth=25,
                                                     min_samples_split=5,
                                                     n_jobs=-1,
                                                     random_state=1234))

In [22]:
# ROC_AUC best training model over all train dataset
y_pred_genres = best_model.predict_proba(X_dtm)
roc_auc_score(y_genres, y_pred_genres, average='macro',multi_class='ovr')

0.9160934694405404

### Precit over test dataframe

In [24]:
# Dataframe to predict
df_test.head()

,year,title,plot
1,1999,Message in a Bottle,"who meets by fate , shall be sealed by fate ...."
4,1978,Midnight Express,"the true story of billy hayes , an american c..."
5,1996,Primal Fear,martin vail left the chicago da ' s office to ...
6,1950,Crisis,husband and wife americans dr . eugene and mr...
7,1959,The Tingler,the coroner and scientist dr . warren chapin ...


In [25]:
# Predict over df_test
X_test_dtm = vect.transform(df_test['plot'])
cols = ['p_Action', 'p_Adventure', 'p_Animation', 'p_Biography', 'p_Comedy', 'p_Crime', 'p_Documentary', 'p_Drama', 'p_Family',
        'p_Fantasy', 'p_Film-Noir', 'p_History', 'p_Horror', 'p_Music', 'p_Musical', 'p_Mystery', 'p_News', 'p_Romance',
        'p_Sci-Fi', 'p_Short', 'p_Sport', 'p_Thriller', 'p_War', 'p_Western']

y_pred=final_model.predict_proba(X_test_dtm)
y_pred=pd.DataFrame(y_pred, index=df_test.index, columns=cols)
y_pred.head()

,p_Action,p_Adventure,p_Animation,p_Biography,p_Comedy,p_Crime,p_Documentary,p_Drama,p_Family,p_Fantasy,...,p_Musical,p_Mystery,p_News,p_Romance,p_Sci-Fi,p_Short,p_Sport,p_Thriller,p_War,p_Western
1,0.074165,0.068195,0.019413,0.030613,0.306419,0.074120,0.047243,0.457376,0.051316,0.096894,...,0.017279,0.051738,0.000000,0.438186,0.040845,0.003888,0.048900,0.141030,0.009716,0.010104
4,0.102886,0.061214,0.018561,0.114983,0.288341,0.187541,0.061513,0.591763,0.052061,0.046554,...,0.017785,0.041038,0.000011,0.127028,0.037793,0.008397,0.012040,0.172882,0.015622,0.010596
5,0.285957,0.091234,0.026353,0.094958,0.354105,0.577770,0.037267,0.664747,0.091540,0.122978,...,0.024694,0.382911,0.000000,0.354433,0.094339,0.013204,0.038756,0.450916,0.049033,0.025647
6,0.151682,0.098810,0.010434,0.033563,0.374941,0.120186,0.028982,0.607940,0.041877,0.056499,...,0.080714,0.098339,0.000000,0.266697,0.131625,0.010209,0.072155,0.264196,0.103851,0.024671
7,0.158420,0.164812,0.019411,0.045229,0.342139,0.329185,0.046160,0.434833,0.126673,0.186819,...,0.014404,0.109639,0.000000,0.162909,0.349235,0.006735,0.021376,0.326584,0.015336,0.017157


In [26]:
# Export
#y_pred.to_csv('pred_genres_text_RF.csv', index_label='ID')